In [3]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import time

from tensorboardX import SummaryWriter
import pandas as pd

mingw_bin = "C:/msys64/mingw64/bin"
os.add_dll_directory(mingw_bin)
import ctypes
lin = ctypes.cdll.LoadLibrary("./cmake-build-debug/liblinear_model.dll")

lin.create_linear_model.argtypes = [ctypes.c_int32]
lin.create_linear_model.restype = ctypes.c_void_p

lin.predict_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_linear_model.restype = ctypes.c_float

lin.release_linear_model.argtypes = [ctypes.c_void_p]
lin.release_linear_model.restype = None

lin.train_linear_model.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_float), ctypes.c_int32, ctypes.c_float, ctypes.c_int32]
lin.train_linear_model.restype = None

lin.create_ovo_classifier.argtypes = [ctypes.c_int32]
lin.create_ovo_classifier.restype = ctypes.c_void_p

lin.train_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float), ctypes.POINTER(ctypes.c_int32), ctypes.c_int32, ctypes.c_int32, ctypes.c_float]
lin.train_ovo_classifier.restype = None

lin.predict_ovo_classifier.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_float)]
lin.predict_ovo_classifier.restype = ctypes.c_int32

lin.release_ovo_classifier.argtypes = [ctypes.c_void_p]
lin.release_ovo_classifier.restype = None


mlp = ctypes.cdll.LoadLibrary("./cmake-build-debug/libmlp.dll") 

mlp.create_mlp_model.argtypes = [ctypes.POINTER(ctypes.c_int), ctypes.c_int]
mlp.create_mlp_model.restype = ctypes.c_void_p

mlp.train_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int,
    ctypes.c_int,
    ctypes.c_float,
    ctypes.c_int,
    ctypes.c_bool,
    ctypes.c_char_p, 
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int
]
mlp.predict_mlp_model.restype = ctypes.POINTER(ctypes.c_float)

mlp.predict_mlp_model.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_bool
]

mlp.release_mlp_model.argtypes = [ctypes.c_void_p]

mlp.get_confusion_matrix.argtypes = [ctypes.c_void_p, ctypes.POINTER(ctypes.c_int32)]
mlp.get_confusion_matrix.restype = None

mlp.evaluate_confusion_matrix.argtypes = [
    ctypes.c_void_p,
    ctypes.POINTER(ctypes.c_float),
    ctypes.POINTER(ctypes.c_float),
    ctypes.c_int,
    ctypes.c_bool 
]
mlp.evaluate_confusion_matrix.restype = None

LOG_FUNC_TYPE = ctypes.CFUNCTYPE(None, ctypes.c_char_p)

@LOG_FUNC_TYPE
def logger_callback(msg):
    print(msg.decode())

In [4]:
classnames_array = ['buffalo', 'elephant', 'zebre']
size = 32

### Changement de résolution des images du dataset:

In [5]:
for classname in classnames_array:
    folder_path = 'images/' + classname 
    for filename in os.listdir(folder_path):
        img = Image.open(folder_path + '/' + filename).convert("RGB")
        img = img.resize(size=(size,size))
        new_folder = './images' + str(size) + '/' + classname
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        img.save(new_folder + '/' + filename, format="JPEG")

### Organisation du Dataset pour l'entrainement

In [6]:
data = []
folder_path = './images32/'
for classname in os.listdir(folder_path):  
    for filename in os.listdir(folder_path + classname):
        img = Image.open(folder_path + classname + '/' + filename)
        
        # Trouver l'index correct dans classnames_array
        index = classnames_array.index(classname)  
        
        one_hot = np.zeros(len(classnames_array), dtype=np.float32)
        one_hot[index] = 1.0
        
        img_array = (np.array(img) / 255.0) - 1.0
        data.append([one_hot, img_array])

data = np.array(data, dtype=object)




### Test avec MLP

In [ ]:
layer_sizes = [size*size*3, 128, 64, len(classnames_array)]
layer_array = (ctypes.c_int * len(layer_sizes))(*layer_sizes)
model = mlp.create_mlp_model(layer_array, len(layer_sizes) - 1)
mlp.set_logger(logger_callback)

np.random.shuffle(data)
split_index = int(len(data) * 0.8)
data_train = data[:split_index]
data_test = data[split_index:]
X_train = np.array([item[1] for item in data_train], dtype=np.float32)
Y_train = np.array([item[0] for item in data_train], dtype=np.float32)
X_test = np.array([item[1] for item in data_test], dtype=np.float32)
Y_test = np.array([item[0] for item in data_test], dtype=np.float32)

X_train_reshaped = X_train.reshape((X_train.shape[0], -1))  # (N, 3072)
X_train_c = X_train_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_train_c = Y_train.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))  # (N, 3072)
X_test_c = X_test_reshaped.ctypes.data_as(ctypes.POINTER(ctypes.c_float))
Y_test_c = Y_test.ctypes.data_as(ctypes.POINTER(ctypes.c_float))

epochs = 1000
learning_rate = 0.01
batch_size = 32

with open("log.csv", "w") as csvfile:
        csvfile.truncate()
        csvfile.write("epoch,accuracy,loss\n")
time.sleep(0.2)

mlp.train_mlp_model(model, X_train_c, Y_train_c, len(data_train), epochs, learning_rate, batch_size, True, b"tanh", X_test_c, Y_test_c, len(data_test))

output_dim = len(classnames_array)
matrix = (ctypes.c_int * (output_dim * output_dim))()
mlp.evaluate_confusion_matrix(model, X_test_c, Y_test_c, len(data_test), True)
mlp.get_confusion_matrix(model, matrix)

mlp.release_mlp_model(model)

# Conversion en numpy array
confusion_np = np.ctypeslib.as_array(matrix).reshape((output_dim, output_dim))
print(confusion_np)
import seaborn as sns

# Matrice de confusion fournie

classnames_array = ['buffalo', 'elephant', 'zebre']

# Affichage avec Seaborn
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
ax = sns.heatmap(confusion_np, annot=True, fmt='d', cmap='Blues', 
                 xticklabels=classnames_array, yticklabels=classnames_array[::-1],
                 linewidths=1, linecolor='gray', cbar=True)

plt.title("Matrice de confusion", fontsize=16)
plt.xlabel("Classe prédite", fontsize=14)
plt.ylabel("Classe réelle", fontsize=14)
plt.tight_layout()
plt.show()

Epoch 10: acc = 39.66%, loss = 2.2564
Epoch 20: acc = 40.08%, loss = 2.0630
Epoch 30: acc = 42.62%, loss = 1.9729
Epoch 40: acc = 44.73%, loss = 1.8311
Epoch 50: acc = 45.99%, loss = 1.7202
Epoch 60: acc = 45.99%, loss = 1.8388
Epoch 70: acc = 42.19%, loss = 1.7618
Epoch 80: acc = 43.46%, loss = 1.7502
Epoch 90: acc = 40.93%, loss = 1.7231
Epoch 100: acc = 45.57%, loss = 1.6764
Epoch 110: acc = 46.84%, loss = 1.6824
Epoch 120: acc = 45.15%, loss = 1.7396
Epoch 130: acc = 43.46%, loss = 1.6946
Epoch 140: acc = 47.68%, loss = 1.6232
Epoch 150: acc = 48.52%, loss = 1.5628
Epoch 160: acc = 45.15%, loss = 1.5279
Epoch 170: acc = 46.84%, loss = 1.5363
Epoch 180: acc = 50.21%, loss = 1.4498
Epoch 190: acc = 47.68%, loss = 1.4507
Epoch 200: acc = 47.26%, loss = 1.4800
Epoch 210: acc = 48.10%, loss = 1.4876
Epoch 220: acc = 45.99%, loss = 1.4918
Epoch 230: acc = 47.26%, loss = 1.5182
Epoch 240: acc = 43.04%, loss = 1.4758
Epoch 250: acc = 43.88%, loss = 1.4559
Epoch 260: acc = 45.57%, loss = 1.

TypeError: object of type 'NoneType' has no len()

<Figure size 800x600 with 0 Axes>

In [ ]:
print(Y_test)


[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0.

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score

X_train_reshaped = X_train.reshape((X_train.shape[0], -1))
X_test_reshaped = X_test.reshape((X_test.shape[0], -1))
Y_train_cat = to_categorical(Y_train, num_classes=len(classnames_array))
Y_test_cat = to_categorical(Y_test, num_classes=len(classnames_array))

def ensure_categorical(Y, num_classes):
    Y = np.array(Y)
    if len(Y.shape) == 1:
        return to_categorical(Y, num_classes)
    elif Y.shape[1] == num_classes:
        return Y 
    else:
        raise ValueError(f"Format inattendu pour Y (shape: {Y.shape})")

Y_train_cat = ensure_categorical(Y_train, len(classnames_array))
Y_test_cat = ensure_categorical(Y_test, len(classnames_array))

model = Sequential([
    Input(shape=(X_train_reshaped.shape[1],)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(len(classnames_array), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_reshaped, Y_train_cat,
          epochs=200,
          batch_size=32,
          validation_data=(X_test_reshaped, Y_test_cat),
          verbose=2)

predictions = model.predict(X_test_reshaped)
pred_labels = np.argmax(predictions, axis=1)
acc = accuracy_score(Y_test, pred_labels)
print(f"\nTest Accuracy: {acc * 100:.2f}%")

Epoch 1/200
30/30 - 1s - 32ms/step - accuracy: 0.3294 - loss: 2.3289 - val_accuracy: 0.3207 - val_loss: 1.2150
Epoch 2/200
30/30 - 0s - 5ms/step - accuracy: 0.4068 - loss: 1.1156 - val_accuracy: 0.5612 - val_loss: 0.9969
Epoch 3/200
30/30 - 0s - 5ms/step - accuracy: 0.4449 - loss: 1.0289 - val_accuracy: 0.5105 - val_loss: 0.9973
Epoch 4/200
30/30 - 0s - 5ms/step - accuracy: 0.4703 - loss: 1.0187 - val_accuracy: 0.5865 - val_loss: 0.9237
Epoch 5/200
30/30 - 0s - 5ms/step - accuracy: 0.5148 - loss: 0.9672 - val_accuracy: 0.5612 - val_loss: 0.8951
Epoch 6/200
30/30 - 0s - 5ms/step - accuracy: 0.5212 - loss: 0.9647 - val_accuracy: 0.5696 - val_loss: 0.9445
Epoch 7/200
30/30 - 0s - 5ms/step - accuracy: 0.5572 - loss: 0.9203 - val_accuracy: 0.5401 - val_loss: 0.9073
Epoch 8/200
30/30 - 0s - 5ms/step - accuracy: 0.5466 - loss: 0.9039 - val_accuracy: 0.6118 - val_loss: 0.8593
Epoch 9/200
30/30 - 0s - 5ms/step - accuracy: 0.5847 - loss: 0.8632 - val_accuracy: 0.5781 - val_loss: 0.8501
Epoch 10/

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

1

In [ ]:
img = Image.open('C:/Users/lgrdp/Downloads/images (10).jpeg').convert("RGB")
img = img.resize(size=(32,32))
img_data = np.array(img) / 255.0

output_array = mlp.predict_mlp_model(model, img_data.ctypes.data_as(ctypes.POINTER(ctypes.c_float)), True)
output_array = ctypes.cast(output_array, ctypes.POINTER(ctypes.c_float * layer_sizes[-1])).contents
output = list(output_array)
class_index = 0
for i in range(len(output)):
    if output[i] == max(output):
        class_index = i
print('predicted class : ' + classnames_array[class_index])
print(output)


predicted class : buffalo
[0.6341931223869324, 0.0993126854300499, 0.2664942145347595]
